In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import nltk
import gensim
import spacy
from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split

import torch as tt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch import autograd

from torchtext.data import Field, LabelField, BucketIterator, ReversibleField, TabularDataset

from gensim.models import KeyedVectors



SEED = 42
np.random.seed(SEED)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
spacy_en = spacy.load('en')
spacy_en.remove_pipe('tagger')
spacy_en.remove_pipe('ner')

def tokenizer(text): # create a tokenizer function
#     return [tok.lemma_ for tok in spacy_en.tokenizer(text) if tok.text.isalpha()]
    return [tok.lemma_ for tok in spacy_en.tokenizer(text)] # not only letters

In [4]:
data = pd.read_csv('../input/train-balanced-sarcasm.csv')

In [5]:
data = data[['label', 'comment']]

In [6]:
data.isnull().values.any()

True

In [7]:
data = data.dropna()

In [8]:
train_dataset, test_dataset = train_test_split(data, test_size=0.2)

In [9]:
train_dataset.to_csv('train_ds.csv', encoding='utf-8', index=False)
test_dataset.to_csv('test_ds.csv', encoding='utf-8', index=False)

In [10]:
classes={
    '0': 0,
    '1': 1
}

TEXT = Field(include_lengths=True, batch_first=True, 
             tokenize=tokenizer,
             eos_token='<eos>',
             lower=True,
             stop_words=nltk.corpus.stopwords.words('english')
            )

LABEL = LabelField(dtype=tt.int64, use_vocab=True, preprocessing=lambda x: classes[x])

train = TabularDataset('train_ds.csv', format='csv', 
                         fields=[('label', LABEL), ('text', TEXT)], 
                         skip_header=True)

test = TabularDataset('test_ds.csv', format='csv', 
                         fields=[('label', LABEL), ('text', TEXT)], 
                         skip_header=True)

In [11]:
TEXT.build_vocab(train, min_freq=5, vectors='glove.6B.300d')
len(TEXT.vocab.itos)

.vector_cache/glove.6B.zip: 862MB [06:34, 2.19MB/s]                               
100%|█████████▉| 399564/400000 [00:46<00:00, 8429.10it/s]

31511

In [12]:
embed_matrix = TEXT.vocab.vectors

100%|█████████▉| 399564/400000 [01:00<00:00, 8429.10it/s]

In [13]:
LABEL.build_vocab(train)

In [14]:
# wv_from_bin = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin', binary=True)
# embed_matrix = tt.FloatTensor(wv_from_bin.vectors)

In [15]:
train, valid = train.split(0.9, stratified=True)

In [22]:
class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, embed_matrix):
        super(MyModel, self).__init__()

        self.embedding = nn.Embedding.from_pretrained(embed_matrix, freeze=False)
        self.rnn = nn.LSTM(input_size=embed_size,
                           hidden_size=hidden_size,
                           bidirectional=True,
                           batch_first=True
                          )
        
        self.fc = nn.Linear(hidden_size * 2 * 2, 512)
        
#         self.i1h = nn.Linear(hidden_size * 2, 256)
#         self.i2h = nn.Linear(hidden_size, 128)
#         self.i2o = nn.Linear(128, 2)
        
#         self.dropout = nn.Dropout(0.2)


#     def init_hidden(self, batch_size, hidden_size):
#         return (autograd.Variable(tt.zeros(1, batch_size, hidden_size)),
#                 autograd.Variable(tt.zeros(1, batch_size, hidden_size)))
        
        
    def forward(self, batch):
        
        x, x_lengths = batch.text
        
        x = x.cuda()
        batch.label = batch.label.cuda()
        
        x = self.embedding(x)

        if x_lengths is not None:
            x_lengths = x_lengths.view(-1).tolist()
            x = nn.utils.rnn.pack_padded_sequence(x, x_lengths, batch_first=True)
            
        _, (hidden, cell) = self.rnn(x)
        
        hidden = hidden.transpose(0,1)
        cell = cell.transpose(0,1)
        hidden = hidden.contiguous().view(hidden.size(0),-1)
        cell = cell.contiguous().view(cell.size(0),-1)
        x = tt.cat([hidden, cell], dim=1).squeeze(1)
        x = self.fc(x)

#         x = F.relu(self.fc(x))
#         x = self.dropout(x)

#         x = F.relu(self.i2h(x))
#         x = self.dropout(x)

#         x = F.relu(self.i1h(x))
#         x = self.dropout(x)

#         x = self.i2o(x)

        return x

In [28]:
tt.cuda.empty_cache()

batch_size = 512

model = MyModel(len(TEXT.vocab.itos),
                embed_size=300,
                embed_matrix=embed_matrix,
                hidden_size=512
               )

model = model.cuda()

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, test),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True
)

optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)
criterion = nn.CrossEntropyLoss()

In [29]:
def calculate_accuracy(predictions, Y):
    max_vals, max_indices = tt.max(predictions.data, 1)
    acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
    
    return acc

In [30]:
def evaluate_test_accuracy(model, test_iterator, criterion):
    epoch_accuracy = 0
    
    model.eval()
    n_batches = len(test_iterator)
    
    with tt.no_grad():
        for batch in test_iterator:
            pred = model(batch)
            accuracy = calculate_accuracy(pred, batch.label)
            epoch_accuracy += accuracy.item()
        
    return epoch_accuracy / n_batches

In [31]:
def _train_epoch(model, iterator, optimizer, criterion, curr_epoch):
    model.train()

    running_loss = 0
    epoch_acc = 0

    n_batches = len(iterator)
    iterator = tqdm_notebook(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)

    for i, batch in enumerate(iterator):
        optimizer.zero_grad()

        pred = model(batch)
        loss = criterion(pred, batch.label)
        loss.backward()
        optimizer.step()

        curr_loss = loss.data.cpu().detach().item()
        
        loss_smoothing = i / (i+1)
        running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss

        iterator.set_postfix(loss='%.5f' % running_loss)
        
        accuracy = calculate_accuracy(pred, batch.label)
        epoch_acc += accuracy.item()

    return running_loss, epoch_acc / n_batches

def _test_epoch(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    
    n_batches = len(iterator)
    with tt.no_grad():
        for batch in iterator:
            pred = model(batch)
            loss = criterion(pred, batch.label)
            epoch_loss += loss.data.item()
            
            accuracy = calculate_accuracy(pred, batch.label)
            epoch_acc += accuracy.item()

    return epoch_loss / n_batches, epoch_acc / n_batches


def nn_train(model, train_iterator, valid_iterator, criterion, optimizer, test_iterator, n_epochs=100,
             scheduler=None, early_stopping=0):

    prev_loss = 100500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    for epoch in range(n_epochs):
        train_loss, train_acc = _train_epoch(model, train_iterator, optimizer, criterion, epoch)
        valid_loss, valid_acc = _test_epoch(model, valid_iterator, criterion)

        valid_loss = valid_loss
        print('validation loss %.5f' % valid_loss)

#         print(f'| Train Loss: {train_loss:.5f} | Train Accuracy: {train_acc:.5f} | Validation Loss: {valid_loss:.5f} | Validation Accuracy: {valid_acc:.5f} |')

        record = {'epoch': epoch, 'train_loss': train_loss, 'valid_loss': valid_loss}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)

In [32]:
nn_train(model, train_iterator, valid_iterator, criterion, optimizer, test_iterator, scheduler=scheduler, 
         n_epochs=10, early_stopping=2)

validation loss 0.55399


validation loss 0.54624


validation loss 0.55383


validation loss 0.58839
Early stopping! best epoch: 1 val 0.54624


In [33]:
def evaluate_test_accuracy(model, test_iterator, criterion):
    epoch_accuracy = 0
    
    model.eval()
    n_batches = len(test_iterator)
    
    with tt.no_grad():
        for batch in test_iterator:
            pred = model(batch)
            accuracy = calculate_accuracy(pred, batch.label)
            epoch_accuracy += accuracy.item()
        
    return epoch_accuracy / n_batches

In [34]:
test_accuracy = evaluate_test_accuracy(model, test_iterator, criterion)

In [ ]:
print('Test Accuracy: %.5f' % test_accuracy)

Test Accuracy: 0.71208
